In [ ]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import feedparser  # for RSS feeds

# --- fetch articles via RSS feed ---
def fetch_rss_articles(feed_url, n=20):
    feed = feedparser.parse(feed_url)
    articles = []
    for entry in feed.entries[:n]:
        if 'summary' in entry and len(entry.summary.split()) > 30:
            articles.append(entry.summary)
        elif 'description' in entry and len(entry.description.split()) > 30:
            articles.append(entry.description)
    return articles

# --- try RSS feeds from multiple sources ---
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/rss.xml",
    "http://feeds.reuters.com/Reuters/worldNews",
    "http://rss.cnn.com/rss/edition_world.rss"
]

articles = []
for feed_url in rss_feeds:
    print(f"Trying RSS feed: {feed_url}")
    art = fetch_rss_articles(feed_url, n=30)
    if art:
        articles.extend(art)
        print(f"  Collected {len(art)} articles")
    if len(articles) >= 30:
        break

# --- fallback if RSS fails ---
if not articles:
    print("RSS fetching failed. Using fallback samples instead.")
    articles = [
        "The economy is showing signs of improvement with inflation coming under control.",
        "A breakthrough in renewable energy offers new hope in the fight against climate change.",
        "Local football team wins the championship, sparking celebrations across the city.",
        "A political scandal is unfolding over leaked documents revealing corruption.",
        "Tech companies are racing to dominate the AI market with new models and services.",
        "Massive protests erupt in the capital demanding electoral reforms.",
        "Stock markets rally as central bank cuts interest rates unexpectedly.",
        "Scientists warn of rising sea levels threatening coastal communities."
    ]

print("Total articles ready for clustering:", len(articles))

# --- vectorize text ---
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(articles)

# --- KMeans clustering ---
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(X)

# --- inspect cluster keywords ---
terms = vectorizer.get_feature_names_out()
cluster_names = {}
for i in range(num_clusters):
    top_indices = np.argsort(kmeans.cluster_centers_[i])[-10:]
    top_terms = [terms[j] for j in top_indices]
    print(f"Cluster {i}: {', '.join(top_terms)}")
    cluster_names[i] = f"Topic {i}: {', '.join(top_terms[:3])}"

# --- classify user input ---
print("\nEnter your own news text to see which cluster it belongs to.")
while True:
    user_input = input("Enter news text (or 'quit'): ")
    if user_input.lower() == 'quit':
        break
    if len(user_input.strip().split()) < 5:
        print("Please enter a longer text snippet.")
        continue
    vec = vectorizer.transform([user_input])
    cluster_id = kmeans.predict(vec)[0]
    print("Matched Cluster →", cluster_names[cluster_id])
    print("-" * 40)


Trying RSS feed: http://feeds.bbci.co.uk/news/world/rss.xml
Trying RSS feed: http://feeds.reuters.com/Reuters/worldNews
Trying RSS feed: http://rss.cnn.com/rss/edition_world.rss
  Collected 7 articles
Total articles ready for clustering: 7
Cluster 0: 70, consent, published, playboy, photograph, nude, magazine, purchased, years, year
Cluster 1: françois, seek, road, recent, search, trip, travel, strip, wonders, world
Cluster 2: routine, stored, auction, pair, game, million, sneakers, air, set, jordan
Cluster 3: color, founders, panicked, payroll, rushed, silicon, stepped, valley, venture, withdraw
Cluster 4: fossil, attempts, promising, sources, stores, shopping, shift, reliance, solution, wean

Enter your own news text to see which cluster it belongs to.
